## Problem 3 - Event based approach

In [27]:
import numpy as np
import pandas as pd

Q1_ARR = "Q1_ARR"
Q1_DONE = "Q1_DONE"
Q2_DONE = "Q2_DONE"

class EventQueue:
    def __init__(self):
        self.queue = {}
        self.passed_events = []
        self.current_time = 0
    
    def plan_q1_arr(self):
        self.queue[self.current_time + 3600 * np.random.exponential(1/100)] = Q1_ARR
    
    def plan_q1_done(self):
        self.queue[self.current_time + 3600 * np.random.exponential(1/200)] = Q1_DONE
    
    def plan_q2_done(self):
        self.queue[self.current_time + 3600 * np.random.exponential(1/110)] = Q2_DONE

    def get_next_event(self):
        # Find next event scheduled in the future
        event_times = sorted(self.queue.keys())
        # Remove events that are in the past or now
        event_times = [t for t in event_times if t > self.current_time]

        # Get the next event
        next_event_time = event_times[0]

        # Get the event type
        event_type = self.queue[next_event_time]

        # Add the event to the passed events
        self.passed_events.append((next_event_time, event_type))
        # Remove the event from the queue
        del self.queue[next_event_time]
        # Update the current time
        self.current_time = next_event_time
        return next_event_time, event_type



def simulate(N=10000):
    queue = EventQueue()
    # Initialize the first event
    queue.plan_q1_arr()
    cars_arrived = 0

    # Initialize the number of cars in each queue
    q1 = 0
    q2 = 0

    while cars_arrived < N:
        time, event_type = queue.get_next_event()

        if event_type == Q1_ARR:
            cars_arrived += 1
            q1 += 1
            if q1 == 1 and q2 < 10:
                queue.plan_q1_done()
            
            queue.plan_q1_arr()

        elif event_type == Q1_DONE:
            q1 -= 1
            q2 += 1

            if q1 != 0 and q2 < 10:
                queue.plan_q1_done()
            
            if q2 == 1:
                queue.plan_q2_done()
        
        elif event_type == Q2_DONE:
            q2 -= 1
            if q2 == 9 and q1 > 0:
                # q2 was just full, but isn't anymore
                # and someone is waiting in q1
                queue.plan_q1_done()
            if q2 != 0:
                queue.plan_q2_done()
    
    return queue

def simulate_df(*args):
    event_queue = simulate(*args)

    df_data = []

    arrived, q1, q2, q1_done, q2_done = 0, 0, 0, 0, 0

    for event_time, event_type in event_queue.passed_events:
        if event_type == Q1_ARR:
            q1 += 1
            arrived += 1
        elif event_type == Q1_DONE:
            q1 -= 1
            q1_done += 1
            q2 += 1
        elif event_type == Q2_DONE:
            q2 -= 1
            q2_done += 1
        
        df_data.append((event_time, event_type, q1, q2, arrived, q1_done, q2_done))

    return pd.DataFrame(df_data, columns=["time", "event_type", "q1", "q2", "arrived", "q1_done", "q2_done"])

simulate_df(10000)

,time,event_type,q1,q2,q1_done,q2_done
0,61.296614,Q1_ARR,1,0,0,0
1,127.825065,Q1_ARR,2,0,0,0
2,151.978744,Q1_DONE,1,1,1,0
3,155.793870,Q1_DONE,0,2,2,0
4,190.823009,Q1_ARR,1,2,2,0
...,...,...,...,...,...,...
29958,356017.897060,Q2_DONE,13,9,9985,9976
29959,356018.018899,Q1_ARR,14,9,9985,9976
29960,356025.722220,Q1_DONE,13,10,9986,9976
29961,356036.799184,Q2_DONE,13,9,9986,9977


In [ ]:
import matplotlib.pyplot as plt
